# LSTM neural network for emergency demand predictions
This notebook contains the code for applying neural network models to smart city data <br>


In [ ]:
# GPU check:

import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
print("Show System RAM Memory: \n \n")
!cat /proc/meminfo | egrep "MemTotal"
device_lib.list_local_devices()

Show System RAM Memory: 
 

MemTotal:       26751672 kB


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6714895522616958429, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7476422979487565813
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5293115004803471463
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9619718780183239234
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
# get additional info about the hardware in the cloud
%cat /proc/cpuinfo
%cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

Tutorial about google colab and GPU access: <br>
https://www.youtube.com/watch?v=f1UK8KPt-KU

In [ ]:
# this allows for accessing files stored in your google drive using the path "/gdrive/My Drive/"
# mounting google drive locally:

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# importing local data to google colab:
from google.colab import files
uploaded = files.upload()

Saving taxi_series_H to taxi_series_H


$\textbf{Background:}$ Tensors are data structures that you can think of as multi-dimensional arrays. Tensors are represented as n-dimensional arrays of base dataypes such as a string or integer -- they provide a way to generalize vectors and matrices to higher dimensions. The shape of a Tensor defines its number of dimensions and the size of each dimension. The rank of a Tensor provides the number of dimensions. Scalars can be used to create 0-d Tensors. Vectors and lists can be used to create 1-d Tensors. Matrices can be used to create 2-d or higher rank Tensors. The shape of a Tensor provides the number of elements in each Tensor dimension.

$\textbf{Neural Networks in Tensorflow:}$ We can also define neural networks in TensorFlow. TensorFlow uses a high-level API called Keras that provides a powerful, intuitive framework for building and training deep learning models. <br> 
Tensors can flow through abstract types called $\textit{Layers}$ -- the building blocks of neural networks. Layers implement common neural networks operations, and are used to update weights, compute losses, and define inter-layer connectivity <br>
<br>
Conveniently, TensorFlow has defined a number of Layers that are commonly used in neural networks, for example a Dense. Now, instead of using a single Layer to define our simple neural network, we'll use the Sequential model from Keras and a single Dense layer to define our network. With the Sequential API, you can readily create neural networks by stacking together layers like building blocks.

# Implementation

In [ ]:
## -- Packages  -- ##

# General
import pandas as pd
import numpy as np

# Time formatting
import datetime

# Load and save data
import pickle
# progress bar
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
#import tikzplotlib as tkz

In [ ]:
##  NN libaries ##
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


## ML libraries ##
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler

### Load data

In [ ]:
#load taxi data. Generated in notebook 'taxi_trips'
filename = '/gdrive/My Drive/Colab Notebooks/emergency_dispatches_bronx_H'
infile = open(filename,'rb')
emergency_ts = pickle.load(infile)
infile.close()

## Preprocessing for 1 month of training data

In [ ]:
## 
emergency_ts_1month = emergency_ts.iloc[(8760 - 1128):-360] 
emergency_ts_1month.shape

(768,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 48 hours ahead
n_test = 24 # output_lags  (changed)


In [ ]:
## Split data in train and test set
train = emergency_ts_1month[0:-n_test]
test = emergency_ts_1month[-n_test:]
print(train.shape)
print(test.shape)

(744,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (661, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (661, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# configuring the inputs for the model
# For Keras, the input has to be in the shape (samples, time steps, features)
# 24 timestep with n features where n is equal to the shape of column [1] of X_train or X_test

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# creating a leaky_relu activation function

def my_leaky_relu(x):
    return tf.nn.leaky_relu(x)

### Fitting the optimal model with all gridsearched parameters

In [ ]:


# fitting the optimal GRU model with the gridsearched hyperparameters
LSTM_model = tf.keras.Sequential()

# Add a GRU layer with 128 units (=dimensionality of the output space = number of neurons)
# option1: return sequences returns the hidden state output for each input time step.
# option2: return state returns the hidden state output and cell state for the last input time step.
# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 128)
LSTM_model.add(tf.keras.layers.LSTM(units=1000, return_sequences = False, input_shape = (1,60), activation=my_leaky_relu, dropout = 0.0))


# output layer 
LSTM_model.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
LSTM_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])
LSTM_model.fit(X_train, Y_train, epochs=500, batch_size=400, verbose=1)
    


Epoch 1/500
2/2 [==============================] - 0s 5ms/step - loss: 10576.4854 - mean_squared_error: 10576.4854
Epoch 2/500
2/2 [==============================] - 0s 6ms/step - loss: 1086.4343 - mean_squared_error: 1086.4343
Epoch 3/500
2/2 [==============================] - 0s 6ms/step - loss: 998.9993 - mean_squared_error: 998.9993
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 1205.8599 - mean_squared_error: 1205.8599
Epoch 5/500
2/2 [==============================] - 0s 6ms/step - loss: 391.9947 - mean_squared_error: 391.9947
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 498.6136 - mean_squared_error: 498.6136
Epoch 7/500
2/2 [==============================] - 0s 6ms/step - loss: 235.9352 - mean_squared_error: 235.9352
Epoch 8/500
2/2 [==============================] - 0s 6ms/step - loss: 214.5300 - mean_squared_error: 214.5300
Epoch 9/500
2/2 [==============================] - 0s 5ms/step - loss: 300.5453 - mean_squared_error: 30

In [ ]:
# making predictions (regular GRU)
y_train_pred = LSTM_model.predict(X_train)
y_test_pred = LSTM_model.predict(X_test)

In [ ]:
# RMSE LSTM forecast
mse_LSTM = mean_squared_error(Y_test, y_test_pred)
sqrt(mse_LSTM)

7.226729087329261

In [ ]:
# MAE LSTM
mae_LSTM = mean_absolute_error(Y_test, y_test_pred)
mae_LSTM

4.135672648747762

## Preprocessing for 6 months of data

In [ ]:
##
emergency_ts_6months = emergency_ts.iloc[(8760 - 4800):-360]
emergency_ts_6months.shape

(4440,)

In [ ]:
## Set paramaters
input_lags = 60 # 2 and a half times the seasonal period
output_lags = 24 # we predict 48 hours ahead
n_test = 24 # output_lags  (changed)

In [ ]:
## Split data in train and test set
train = emergency_ts_6months[0:-n_test]
test = emergency_ts_6months[-n_test:]
print(train.shape)
print(test.shape)

(4416,)
(24,)


In [ ]:
## Create lagged values for both input and output window (24)
data = train.copy()
n_train = len(data)

##Create lagged values for input
df = pd.DataFrame()
for i in range(input_lags,0,-1):
    df['t-' + str(i)] = data.shift(i)

##Create lagged values for output
for j in range(0,output_lags,1):
    df['t+' + str(j)] = data.shift(-j)
    
df = df[input_lags:(n_train-output_lags+1)]

In [ ]:
## splitting the training set into labels and features
X_train = df.iloc[:,:input_lags] # from the beginning to input_lags
Y_train = df.iloc[:,input_lags:] # from input_lags to the end

## Use the last window of the training set as the features for the test set. This requires a combination of 
## X_train and Y_train.
X_test = X_train.iloc[len(X_train) - 1,:][output_lags:]
X_test = X_test.append(Y_train.iloc[len(Y_train) - 1,:]).values.reshape(1,input_lags)
Y_test = test[:output_lags].values.reshape(1,output_lags)

X_train = X_train.values # 54 steps back (54 lags)
Y_train = Y_train.values # 24 steps ahead

print("X_train: " + "type: " + str(type(X_train)) + "\tshape: " + str(X_train.shape))
print("Y_train: " + "type: " + str(type(Y_train)) + "\tshape: " + str(Y_train.shape))
print("X_test: " + "type: " + str(type(X_test)) + "\tshape: " + str(X_test.shape))
print("Y_test: " + "type: " + str(type(Y_test)) + "\tshape: " + str(Y_test.shape))

X_train: type: <class 'numpy.ndarray'>	shape: (4333, 60)
Y_train: type: <class 'numpy.ndarray'>	shape: (4333, 24)
X_test: type: <class 'numpy.ndarray'>	shape: (1, 60)
Y_test: type: <class 'numpy.ndarray'>	shape: (1, 24)


In [ ]:
# configuring the inputs for the model
# For Keras, the input has to be in the shape (samples, time steps, features)
# 24 timestep with n features where n is equal to the shape of column [1] of X_train or X_test

X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

### Fitting the optimal model with all gridsearched parameters

In [ ]:


# fitting the optimal GRU model with the gridsearched hyperparameters
LSTM_model = tf.keras.Sequential()

# Add a GRU layer with 128 units (=dimensionality of the output space = number of neurons)
# option1: return sequences returns the hidden state output for each input time step.
# option2: return state returns the hidden state output and cell state for the last input time step.
# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 128)
LSTM_model.add(tf.keras.layers.LSTM(units=1000, return_sequences = False, input_shape = (1,60), activation=my_leaky_relu, dropout = 0.0))


# output layer 
LSTM_model.add(tf.keras.layers.Dense(24))
# the compile() method configures the model for training
LSTM_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate = 0.01),
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])
LSTM_model.fit(X_train, Y_train, epochs=500, batch_size=400, verbose=1)
    


Epoch 1/500
11/11 [==============================] - 0s 8ms/step - loss: 1258.6895 - mean_squared_error: 1258.6895
Epoch 2/500
11/11 [==============================] - 0s 7ms/step - loss: 134.3596 - mean_squared_error: 134.3596
Epoch 3/500
11/11 [==============================] - 0s 8ms/step - loss: 92.9163 - mean_squared_error: 92.9163
Epoch 4/500
11/11 [==============================] - 0s 7ms/step - loss: 77.5659 - mean_squared_error: 77.5659
Epoch 5/500
11/11 [==============================] - 0s 7ms/step - loss: 73.4895 - mean_squared_error: 73.4896
Epoch 6/500
11/11 [==============================] - 0s 7ms/step - loss: 71.1869 - mean_squared_error: 71.1869
Epoch 7/500
11/11 [==============================] - 0s 7ms/step - loss: 68.0964 - mean_squared_error: 68.0964
Epoch 8/500
11/11 [==============================] - 0s 7ms/step - loss: 67.2176 - mean_squared_error: 67.2176
Epoch 9/500
11/11 [==============================] - 0s 7ms/step - loss: 66.2759 - mean_squared_error: 66.

In [ ]:
# making predictions (regular GRU)
y_train_pred = LSTM_model.predict(X_train)
y_test_pred = LSTM_model.predict(X_test)

In [ ]:
# RMSE LSTM forecast
mse_LSTM = mean_squared_error(Y_test, y_test_pred)
sqrt(mse_LSTM)

5.780849396779729

In [ ]:
# MAE LSTM
mae_LSTM = mean_absolute_error(Y_test, y_test_pred)
mae_LSTM

4.135672648747762